In [75]:
import xlwings as xw
import openpyxl
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

In [76]:
### Create a copy and instantiate all values
def init():
    original_file = "../test-invoices/General ledger.xlsx"
    new_file = "../results/testing.xlsx"

    original_wb = xw.Book(original_file)
    new_wb = xw.Book(new_file)

    wb = openpyxl.load_workbook('../results/testing.xlsx') 
    count = len(wb.sheetnames)

    # Only copy and instantiate when creating from scratch
    # Prevents data loss
    if wb.active.title != "Updated":
        original_ws = original_wb.sheets(1)
        original_ws.api.Copy(After=new_wb.sheets(1).api)
        new_wb.sheets(2).name = "Updated"

        new_wb.sheets(2).range("A10:M18").delete()
        new_wb.sheets(2).range("A10:M50").font.name = "Arial"
        new_wb.sheets(2).range("A10:M50").font.size = 8
        new_wb.sheets(2).range("K10:M50").number_format = "0.00"
        new_wb.save()

def store_values(row, number, date, recipient, desc, amount):

    file_path = "../results/testing.xlsx"
    wb = xw.Book(file_path)

    ws = wb.sheets["Updated"]

    if date is not None:
        ws[f"A{row}"].value = date.value

    ws[f"B{row}"].value = "I"
    
    if number is not None:
        ws[f"C{row}"].value = number.value

    if desc is not None:
        ws[f"E{row}"].value = desc.value

    if recipient is not None:
        ws[f"F{row}"].value = recipient.value

    if amount is not None:
        ws[f"K{row}"].value = amount.value.amount

    wb.save()

In [77]:
def model():
    # Store connection information  
    endpoint = "https://docintfeng.cognitiveservices.azure.com/"
    key = "Dcg3SI88GLqCcqxspAEJrrZ9A9YhZntJ7oV33e3BwG1oJ4XRvFIgJQQJ99ALACYeBjFXJ3w3AAALACOGy3Vg"

    fileUri = "https://github.com/YggdrasilSacred/DocToExcel/blob/main/Labfiles/01-prebuild-models/test-invoices/sample-invoice.pdf?raw=true"
    fileLocale = "en-US"
    fileModelId = "prebuilt-invoice"

    print(f"\nConnecting to Forms Recognizer at: {endpoint}")
    print(f"Analyzing invoice at: {fileUri}")

    # Create the client

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    # Analyse the invoice

    poller = document_analysis_client.begin_analyze_document_from_url(
        fileModelId, fileUri, locale=fileLocale 
    )
    invoices = poller.result()

    # Display invoice information to the user

    rowcount = 10

    for idx, invoice in enumerate(invoices.documents):
        print("\n--------Recognizing invoice #{}--------\n".format(idx + 1))

        file_path = "../results/testing.xlsx"
        wb = openpyxl.load_workbook(file_path)
        ws = wb["Updated"]
        
        ### ----Invoice header details---- ###

        # Invoice number
        invoice_number = invoice.fields.get("InvoiceId")
        if invoice_number:
            print(
                "Invoice Number: {} has confidence: {}".format(
                    invoice_number.value, invoice_number.confidence
                )
            )

        # Invoice date
        invoice_date = invoice.fields.get("InvoiceDate")
        if invoice_date:
            print(
                "Invoice Date: {} has confidence: {}".format(
                    invoice_date.value, invoice_date.confidence
                )
            )

        ### ----Billing Information---- ###

        # Bill Recipient
        billing_address_recipient = invoice.fields.get("BillingAddressRecipient")
        if billing_address_recipient:
            print(
                "Billing Address Recipient: {} has confidence: {}".format(
                    billing_address_recipient.value,
                    billing_address_recipient.confidence,
                )
            )

        ### ----Service/Item Details---- ###

        print("Invoice items:")

        for idx, item in enumerate(invoice.fields.get("Items").value):
            print("...Item #{}".format(idx + 1))

            # Description
            item_description = item.value.get("Description")    
            if item_description:
                print(
                    "......Description: {} has confidence: {}".format(
                        item_description.value, item_description.confidence
                    )
                )

            # Item total amount
            amount = item.value.get("Amount")
            if amount:
                print(
                    "......Amount: {} has confidence: {}".format(
                        amount.value, amount.confidence
                    )
                )

            store_values(rowcount, invoice_number, invoice_date, billing_address_recipient, item_description, amount)

            rowcount += 1

        print("----------------------------------------")
        print("\nNow storing values...")


    print("\nAnalysis complete.\n")

In [78]:
init()
model()


Connecting to Forms Recognizer at: https://docintfeng.cognitiveservices.azure.com/
Analyzing invoice at: https://github.com/YggdrasilSacred/DocToExcel/blob/main/Labfiles/01-prebuild-models/test-invoices/sample-invoice.pdf?raw=true

--------Recognizing invoice #1--------

Invoice Number: IN23/12651 has confidence: 0.958
Invoice Date: 2023-04-15 has confidence: 0.935
Billing Address Recipient: Infinity Wave Sdn Bhd has confidence: 0.921
Invoice items:
...Item #1
......Description: Cleaning Service 4 Hrs 2 Persons has confidence: 0.937
......Amount: RM180.0 has confidence: 0.937
----------------------------------------

Now storing values...

Analysis complete.

